In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn.decomposition import FastICA
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot

In [2]:
# acusition data
#  veg12 - session 2 - 34 answers (17 yes 17 no)
# marker_flag = 0
# I = pd.read_csv('IHE-01-11.02.19-15.11.16_YES_NO.csv', skiprows=1, header=None);
# veg12 session 3 - 16 Questions (8 yes 8 no)
marker_flag = 0
I = pd.read_csv('IHE-002_yesno-28.02.19-12.03.26.csv', skiprows=1, header=None);
# big_df = pd.concat([I, I1, I2, I3,...], axis=1)
# veg12 - session 4.2 - 35 answers (14 yes 21 no)
# marker_flag = 0
# I1 = pd.read_csv('yes_no_1.csv', skiprows=1, header=None);
# I2 = pd.read_csv('yes_no_2.csv', skiprows=1, header=None);
# I3 = pd.read_csv('yes_no_3.csv', skiprows=1, header=None);
# I = pd.concat([I1, I2, I3], axis=1)
# veg14 - session 1 - 34 answers (15 yes 19 no)
# marker_flag = 0
# I1 = pd.read_csv('VEG14-VEG14001-16.12.19-16.03.10.csv', skiprows=1, header=None);
# I2 = pd.read_csv('VEG14-VEG14002-16.12.19-16.06.32.csv', skiprows=1, header=None);
# I3 = pd.read_csv('VEG14-VEG14003-16.12.19-16.16.18.csv', skiprows=1, header=None);
# I = pd.concat([I1, I2, I3], axis=1)
# I = ?
# veg16
# marker_flag = 1

In [3]:
len(I)

62720

In [4]:
I.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39],
           dtype='int64')

In [5]:
# Raw EEG 14 electrode 
EEG_Elec_data = I.iloc[:, 2:16]
# EEG_Elec_data.shape
EEG_data = EEG_Elec_data.to_numpy()
#print(np.size(EEG_data))
# print(len(EEG_data))
# marker
if marker_flag == 0:
    marker = I[19]
    # print(marker)
elif marker_flag == 1:
    marker = I[21]

In [6]:
len(EEG_data[0])
# len(EEG_data[:,1])

14

In [7]:
# EEG_data.shape
# np.transpose(EEG_data)
EEG_data.shape

(62720, 14)

In [8]:
def normy(a):
  b = (a-np.mean(a))/(np.std(a))
  return b

In [9]:
NumOfElectrodes = 14
for i in range(1,NumOfElectrodes+1):
  #print(i)
  EEG_data[:,i-1] = normy(EEG_data[:,i-1])

In [ ]:
# check if the normalization is on the each electrode
# np.mean(EEG_data[:,0])
# np.std(EEG_data[:,0])

In [10]:
ica = FastICA(n_components=NumOfElectrodes)
EEG_data = ica.fit_transform(EEG_data)  # Reconstruct signals
mixing_matrix = ica.mixing_  # Get estimated mixing matrix

In [11]:
EEG_data.shape

(62720, 14)

In [12]:
# Find the non-zero indexes of the maker
# print(np.nonzero(np.transpose(marker.to_numpy()))[0])
marker_indexs = np.nonzero(np.transpose(marker.to_numpy()))[0]
val_marker = marker[marker_indexs]
print(val_marker)
NumOfAnswers = np.sum(val_marker==127) + np.sum(val_marker==128)
# print(NumOfAnswers)
val_marker_1 = val_marker[1:len(val_marker)]
val_marker_2 = val_marker[0:len(val_marker)-1]
Answers_excel_labels = val_marker_1[((val_marker_1==1) | (val_marker_1==2)) & ((val_marker_2!=127) & (val_marker_2!=128))]
# print(Answers_excel_labels)
Answers_0_1_labels = Answers_excel_labels - 1
Answers_0_1_labels = Answers_0_1_labels.to_numpy()
print(Answers_0_1_labels)
# print(type(Answers_0_1_labels.to_numpy()[0]))
# Number of "Yes" and "No" Answers
Num_of_Yes_Answers = np.sum(Answers_excel_labels==2)
# print(Num_of_Yes_Answers)
Num_of_No_Answers = np.sum(Answers_excel_labels==1)
# print(Num_of_No_Answers)
# arrange the answers in array
k=0
j=0
# parametres to build the 4 seconds time-window
num_samp = 128                   # number of delay samples before the question finished
m = 1+2*num_samp/128             # shift for m seconds.
shift = -num_samp/128            # in seconds
NumOfElectrodes = 14
CLIS_Answers_Rows = int(NumOfAnswers*NumOfElectrodes)
CLIS_Answers_Cols = int(128*m+2)
# CLIS_answers = np.ones((CLIS_Answers_Rows,CLIS_Answers_Cols))
CLIS_answers = np.empty([CLIS_Answers_Rows, CLIS_Answers_Cols], dtype = float)
print(type(CLIS_answers))
# CLIS_answers.shape

2088      24.0
2282       1.0
2285      87.0
2312      87.0
2314      87.0
         ...  
60246     82.0
60258     82.0
60283     82.0
60315     82.0
60346    127.0
Name: 19, Length: 209, dtype: float64
[0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.]
<class 'numpy.ndarray'>


In [13]:
print(type(EEG_data))
EEG_data.shape
print(type(CLIS_answers))
CLIS_answers.shape
print(EEG_data.ndim)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
2


In [14]:
print(EEG_data[0, :384].shape)
print(CLIS_answers[0, :384].shape)
print(EEG_data.shape)
print(np.transpose(EEG_data).shape)
print(CLIS_answers.shape)
EEG_data[1, 0:384]
CLIS_answers[0,:384] = EEG_data.T[0, :384]
# = EEG_data[0, 0:384]
# print(EEG_data[13, ].ndim)
# print(len(EEG_data[13, :]))
# print(EEG_data.ndim)
# EEG_data = np.transpose(EEG_data)
# CLIS_answers[0,0:385] = EEG_data[13, 62592:62976]
# CLIS_answers[0,0] = EEG_data[13, 62592]
# print(len(EEG_data[13, :]))

(14,)
(384,)
(62720, 14)
(14, 62720)
(224, 386)


In [15]:
for l in range(1,NumOfElectrodes+1):
   print(l)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [16]:
m = int(m)
shift = int(shift)
print(Answers_0_1_labels,type(Answers_0_1_labels))
for i in range(len(marker)):
  if (marker[i] == 127 or marker[i] == 128):
      for l in range(1,NumOfElectrodes+1):
          # answers are arranged that rows correspond to observations and
          # columns are the samples and any block of 14 rows is one
          # question by 14 channles(electrodes),and the last column
          # is the lables
          CLIS_answers[k+l-1,0:128*m] = EEG_data[(i+128*shift-1):(i+128*m+128*shift-1), l-1]
          CLIS_answers[k+l-1,128*m+1] = Answers_0_1_labels[j]
      # adding the labeles
      CLIS_answers[k,128*m+1] = Answers_0_1_labels[j]
      CLIS_answers[k+1,128*m+1] = Answers_0_1_labels[j]
      CLIS_answers[k+2,128*m+1] = Answers_0_1_labels[j]
      CLIS_answers[k+3,128*m+1] = Answers_0_1_labels[j]
      j = j + 1
      k = k + NumOfElectrodes

[0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.] <class 'numpy.ndarray'>


In [17]:
print(len(CLIS_answers[1,:]))

386


In [18]:
def IQR(dist):
    return np.percentile(dist, 75) - np.percentile(dist, 25)

In [19]:
NumOfFeatures = 10
# print(NumOfAnswers, type(NumOfAnswers))
NumOfAnswers = int(NumOfAnswers)
# print(NumOfAnswers, type(NumOfAnswers))
# print(NumOfElectrodes, type(NumOfElectrodes))
# print(NumOfFeatures,type(NumOfFeatures))
CLIS_Features = np.empty([NumOfAnswers*NumOfElectrodes, NumOfFeatures], dtype = float)
EndCols = len(CLIS_answers[1,:]) - 1
for f in range(NumOfAnswers*NumOfElectrodes):
  CLIS_Features[f,0] = np.min(CLIS_answers[f,EndCols-1])
  CLIS_Features[f,1] = np.var(CLIS_answers[f,EndCols-1])
  CLIS_Features[f,2] = np.max(CLIS_answers[f,EndCols-1])
  CLIS_Features[f,3] = np.mean(CLIS_answers[f,EndCols-1])
  CLIS_Features[f,4] = skew(CLIS_answers[f,EndCols-1])
  CLIS_Features[f,5] = np.percentile(CLIS_answers[f,EndCols-1],25)
  CLIS_Features[f,6] = np.percentile(CLIS_answers[f,EndCols-1],50)
  CLIS_Features[f,7] = IQR(CLIS_answers[f,EndCols-1])
  CLIS_Features[f,8] = kurtosis(CLIS_answers[f,EndCols-1])
  # The last feature is the labels 
  CLIS_Features[f,9] = CLIS_answers[f,EndCols]

# print(CLIS_Features.shape)

In [20]:
CLIS_Features.shape

(224, 10)

In [21]:
int(len(CLIS_Features[:,1])/NumOfAnswers)

14

In [22]:
print(NumOfFeatures-2)

8


In [23]:
# labels columbn
print(CLIS_Features[:,9])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [24]:
answers_idx_vector =  np.linspace(1,NumOfAnswers,NumOfAnswers)
answers_idx_vector

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16.])

In [54]:
X_train_idx, X_test_idx, y_train, y_test = train_test_split(answers_idx_vector, Answers_0_1_labels, test_size=0.4, random_state=42)

In [55]:
cols = np.linspace(1,NumOfFeatures-1,NumOfFeatures-1)
cols = cols - 1
cols = cols.astype(int)

In [56]:
X_train_idx = X_train_idx.astype(int)
X_test_idx = X_test_idx.astype(int)
X_train_idx = np.sort(X_train_idx)

In [59]:
CLIS_Features_orded_by_answers = np.empty([NumOfAnswers*NumOfElectrodes, NumOfFeatures-1], dtype = float)
X_train = []
for e in range(NumOfElectrodes): 
      # CLIS_Features_orded_by_answers[,:] = CLIS_Features

0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [48]:
# CLIS_Features.shape
# X_train = [];        # empty list
# for X_train_idx in CLIS_Features[:,:-1]:
  #X_train.append([X_train_idx])  

[[1827.    0. 1827. ...    0.   -3.    0.]
 [1830.    0. 1830. ...    0.   -3.    0.]
 [1833.    0. 1833. ...    0.   -3.    0.]
 ...
 [   4.    0.    4. ...    0.   -3.    0.]
 [   4.    0.    4. ...    0.   -3.    0.]
 [   4.    0.    4. ...    0.   -3.    0.]]


In [28]:


# X_train = CLIS_Features[][cols]
# X_test = CLIS_Features[X_test_idx][cols]

In [30]:
A = [[1, 4, 5, 12], 
    [-5, 8, 9, 0],
    [-6, 7, 11, 19]]
print(A)

[[1, 4, 5, 12], [-5, 8, 9, 0], [-6, 7, 11, 19]]


In [ ]:
X_train_idx
# len(Answers_0_1_labels)
# y_train_idx
# len(X_train_idx)

array([10.,  3., 16.,  5.,  8., 11., 13.,  4.,  7.])

In [ ]:
X_test_idx

array([ 1.,  2.,  6., 15., 14., 12.,  9.])

In [ ]:
# LDA
# sklearn_lda = LDA(n_components=1)
# X_lda_sklearn = sklearn_lda.fit_transform(X, y)

for e in range(1,NumOfElectrodes+1):
  # LDA
  sklearn_lda = LDA(n_components=1)
  X_lda_sklearn = sklearn_lda.fit_transform(X_train[,], y_train[,])
